In [1]:
import sys
sys.path.insert(0, '../')

import random
import numpy as np
import pandas as pd
from src.features.new_or_used import build_dataset
from src.features.data_preprocess_funct import categorize_data_types, column_unique_values, count_column_unique_values, column_unique_keys, missing_values_table, preprocess_str_columns

## Import Data

In [2]:
# Import the data and convert it into dataframes for easier manipulation.
X_train, y_train, X_test, y_test = build_dataset()

X_train = pd.DataFrame(X_train)
X_test = pd.DataFrame(X_test)
y_train = pd.DataFrame(y_train)
y_test = pd.DataFrame(y_test)

## Data Preprocess

In [3]:
# Evaluate if there are unique columns in the training and evaluation dataframes.
train_columns = set(X_train.columns)
test_columns = set(X_test.columns)

print("Number of training columns:\t", len(train_columns))
print("Number of test columns:\t\t", len(test_columns))

train_columns_unique = train_columns - test_columns

print(f"Unique training columns.: {train_columns_unique}")

Number of training columns:	 45
Number of test columns:		 44
Unique training columns.: {'condition'}


The only difference between the training and test data columns is the "condition" column.

In [4]:
# Rename variable for easier workflow.
df = X_train
pd.set_option('display.max_columns', None)
df.head(5)

,seller_address,warranty,sub_status,condition,deal_ids,base_price,shipping,non_mercado_pago_payment_methods,seller_id,variations,site_id,listing_type_id,price,attributes,buying_mode,tags,listing_source,parent_item_id,coverage_areas,category_id,descriptions,last_updated,international_delivery_mode,pictures,id,official_store_id,differential_pricing,accepts_mercadopago,original_price,currency_id,thumbnail,title,automatic_relist,date_created,secure_thumbnail,stop_time,status,video_id,catalog_product_id,subtitle,initial_quantity,start_time,permalink,sold_quantity,available_quantity
0,"{'country': {'name': 'Argentina', 'id': 'AR'},...",None,[],new,[],80.0,"{'local_pick_up': True, 'methods': [], 'tags':...","[{'description': 'Transferencia bancaria', 'id...",8208882349,[],MLA,bronze,80.0,[],buy_it_now,[dragged_bids_and_visits],,MLA6553902747,[],MLA126406,[{'id': 'MLA4695330653-912855983'}],2015-09-05T20:42:58.000Z,none,"[{'size': '500x375', 'secure_url': 'https://a2...",MLA4695330653,NaN,None,True,NaN,ARS,http://mla-s1-p.mlstatic.com/5386-MLA469533065...,Auriculares Samsung Originales Manos Libres Ca...,False,2015-09-05T20:42:53.000Z,https://a248.e.akamai.net/mla-s1-p.mlstatic.co...,1446669773000,active,None,NaN,None,1,1441485773000,http://articulo.mercadolibre.com.ar/MLA4695330...,0,1
1,"{'country': {'name': 'Argentina', 'id': 'AR'},...",NUESTRA REPUTACION,[],used,[],2650.0,"{'local_pick_up': True, 'methods': [], 'tags':...","[{'description': 'Transferencia bancaria', 'id...",8141699488,[],MLA,silver,2650.0,[],buy_it_now,[],,MLA7727150374,[],MLA10267,[{'id': 'MLA7160447179-930764806'}],2015-09-26T18:08:34.000Z,none,"[{'size': '499x334', 'secure_url': 'https://a2...",MLA7160447179,NaN,None,True,NaN,ARS,http://mla-s1-p.mlstatic.com/23223-MLA71604471...,Cuchillo Daga Acero Carbón Casco Yelmo Solinge...,False,2015-09-26T18:08:30.000Z,https://a248.e.akamai.net/mla-s1-p.mlstatic.co...,1448474910000,active,None,NaN,None,1,1443290910000,http://articulo.mercadolibre.com.ar/MLA7160447...,0,1
2,"{'country': {'name': 'Argentina', 'id': 'AR'},...",None,[],used,[],60.0,"{'local_pick_up': True, 'methods': [], 'tags':...","[{'description': 'Transferencia bancaria', 'id...",8386096505,[],MLA,bronze,60.0,[],buy_it_now,[dragged_bids_and_visits],,MLA6561247998,[],MLA1227,[{'id': 'MLA7367189936-916478256'}],2015-09-09T23:57:10.000Z,none,"[{'size': '375x500', 'secure_url': 'https://a2...",MLA7367189936,NaN,None,True,NaN,ARS,http://mla-s1-p.mlstatic.com/22076-MLA73671899...,"Antigua Revista Billiken, N° 1826, Año 1954",False,2015-09-09T23:57:07.000Z,https://a248.e.akamai.net/mla-s1-p.mlstatic.co...,1447027027000,active,None,NaN,None,1,1441843027000,http://articulo.mercadolibre.com.ar/MLA7367189...,0,1
3,"{'country': {'name': 'Argentina', 'id': 'AR'},...",None,[],new,[],580.0,"{'local_pick_up': True, 'methods': [], 'tags':...","[{'description': 'Transferencia bancaria', 'id...",5377752182,[],MLA,silver,580.0,[],buy_it_now,[],,None,[],MLA86345,[{'id': 'MLA9191625553-932309698'}],2015-10-05T16:03:50.306Z,none,"[{'size': '441x423', 'secure_url': 'https://a2...",MLA9191625553,NaN,None,True,NaN,ARS,http://mla-s2-p.mlstatic.com/183901-MLA9191625...,Alarma Guardtex Gx412 Seguridad Para El Automo...,False,2015-09-28T18:47:56.000Z,https://a248.e.akamai.net/mla-s2-p.mlstatic.co...,1449191596000,active,None,NaN,None,1,1443466076000,http://articulo.mercadolibre.com.ar/MLA9191625...,0,1
4,"{'country': {'name': 'Argentina', 'id': 'AR'},...",MI REPUTACION.,[],used,[],30.0,"{'local_pick_up': True, 'methods': [], 'tags':...","[{'description': 'Transferencia bancaria', 'id...",2938071313,[],MLA,bronze,30.0,[],buy_it_now,[dragged_bids_and_visits],,MLA3133256685,[],MLA41287,[{'id': 'MLA7787961817-902981678'}],2015-08-28T13:37:41.000Z,none,"[{'size': '375x500', 'secure_url': 'https://a2...",MLA7787961817,NaN,None,True,NaN,ARS,http://mla-s2-p.mlstatic.com/13595-MLA77879618...,Serenata - Jennifer Blake,False,2015-08-24T22:07:20.000Z,https://a248.e.akamai.net/mla-s2-p.mlstatic.co

We obtain the types of objects present in each column of the dataframe.

In [5]:
types_df = categorize_data_types(df)

First, we will focus on working with columns composed of iterable types, as these can be unpacked into multiple lists, thus facilitating data manipulation.

### Dictionary columns

In [6]:
columns_w_dict = types_df[types_df['basic_types'].str.contains("dict")].index.tolist()
print(f"dict columns:\t",columns_w_dict)

dict columns:	 ['seller_address', 'shipping']


#### seller_address

First, we retrieve all the unique keys present in the dictionaries within the column 'seller_address'

In [7]:
print(f"Unique keys in seller_address: {column_unique_keys(df['seller_address'])}")

Unique keys in seller_address: {'country', 'city', 'state'}


Unpack 'seller_address' into 'seller_country', 'seller_state', and 'seller_city', then remove 'seller_address'.

In [8]:
# STEP 01: Unpack and delete seller_address 
df['seller_country'] = df.apply(lambda x : x['seller_address']['country']['name'], axis = 1)
df['seller_state'] = df.apply(lambda x : x['seller_address']['state']['name'], axis = 1)
df['seller_city'] = df.apply(lambda x : x['seller_address']['city']['name'], axis = 1)

df = df.drop(columns=["seller_address"])

#### shipping

First, we retrieve all the unique keys present in the dictionaries within the column 'shipping'

In [9]:
print(f"Unique keys in shipping: {column_unique_keys(df['shipping'])}")

Unique keys in shipping: {'mode', 'tags', 'free_methods', 'local_pick_up', 'free_shipping', 'methods', 'dimensions'}


Unpack 'shipping' into 'shipping_dimensions', 'free_shipping', 'local_pick_up', 'shipping_methods', 'shipping_free_methods', 'shipping_mode', and 'shipping_tags', then delete 'shipping'.

In [10]:
# STEP 02: Unpack and delete shipping
df['shipping_dimensions'] = df.apply(lambda x : x['shipping'].get('dimensions', None), axis = 1)
df['free_shipping'] = df.apply(lambda x : x['shipping'].get('free_shipping', None), axis = 1)
df['local_pick_up'] = df.apply(lambda x : x['shipping'].get('local_pick_up', None), axis = 1)
df['shipping_methods'] = df.apply(lambda x : x['shipping'].get('methods', None), axis = 1) # Will be deleted later
df['shipping_free_methods'] = df.apply(lambda x : x['shipping'].get('free_methods', None), axis = 1) # Will be deleted later
df['shipping_mode'] = df.apply(lambda x : x['shipping'].get('mode', None), axis = 1)
df['shipping_tags'] = df.apply(lambda x : x['shipping'].get('tags', None), axis = 1) # Will be deleted later

df = df.drop(columns=["shipping"])

Update the types of objects present in each column of the dataframe.

In [11]:
types_df = categorize_data_types(df)

Next, we will focus on the columns comprised of lists, as these are more challenging to unpack and require a more detailed inspection to fully understand their contents and implications.

### List Columns

In [12]:
columns_w_list = types_df[types_df['basic_types'].str.contains("list")].index.tolist()
print(f"list columns:\t",columns_w_list)

list columns:	 ['sub_status', 'deal_ids', 'non_mercado_pago_payment_methods', 'variations', 'attributes', 'tags', 'coverage_areas', 'descriptions', 'pictures', 'shipping_methods', 'shipping_free_methods', 'shipping_tags']


Let's examine the lengths of the lists within the columns as this can provide us with a better understanding of how to handle the data.

In [13]:
if False:
    for column in columns_w_list:
        lengths = set(df[column].apply(lambda x: len(x) if isinstance(x, list) else None))
        print(f"Lengths of {column}",lengths)

Lengths of sub_status {0, 1}
Lengths of deal_ids {0, 1}
Lengths of non_mercado_pago_payment_methods {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12}
Lengths of variations {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 30, 34, 35, 36, 42, 50}
Lengths of attributes {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 26, 27, 29, 31, 36, 37, 38, 39, 40, 41, 42, 45, 46, 47, 48, 50, 52, 54, 63, 64, 65, 66, 67, 71, 73, 75, 77, 78, 81}
Lengths of tags {0, 1, 2}
Lengths of coverage_areas {0}
Lengths of descriptions {0, 1}
Lengths of pictures {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 18, 19, 20, 21, 22, 23, 24, 25, 27, 28, 29, 30, 33, 34, 36}
Lengths of shipping_methods {0.0, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, na

#### coverage_areas, sub_status, deal_ids, descriptions, shipping_tags and shipping_methods

Unpack columns containing lists with a length of up to 1.

In [14]:
# STEP 03: Unpack ists with a length of up to 1.
columns_to_unpack = ["coverage_areas", "sub_status", "deal_ids", "descriptions", "shipping_tags","shipping_methods"]

for column in columns_to_unpack:
    df[column] = df[column].apply(lambda x: x[0] if isinstance(x, list) and len(x) > 0 else None)

It seems that the 'descriptions' column contains dictionaries in string format. We need to convert them back to dictionaries for further processing.

In [15]:
# STEP 04: Unpack "descriptions"
df['descriptions'] = df['descriptions'].apply(lambda x: eval(x) if x is not None else None)

#### shipping_free_methods 

The 'shipping_free_methods' column is nearly empty and does not justify unpacking. It will be removed in a subsequent step.

In [16]:
count_column_unique_values(df["shipping_free_methods"])

{'None': 87311,
 "[{'rule': {'value': None, 'free_mode': 'country'}, 'id': 73328}]": 2641,
 "[{'rule': {'value': None, 'free_mode': 'country'}, 'id': 73330}]": 37,
 "[{'rule': {'value': None, 'free_mode': 'country'}, 'id': 501145}]": 6,
 "[{'rule': {'value': None, 'free_mode': 'country'}, 'id': 501146}]": 4,
 "[{'rule': {'value': None, 'free_mode': 'country'}, 'id': 73328}, {'rule': {'value': None, 'free_mode': 'country'}, 'id': 73330}]": 1}

#### tags

Unpack the 'tags' into boolean columns and then remove the 'tags' column.

In [17]:
# STEP 05: Unpack tags in boolean columns and delete tags
unique_tags = set(item for sublist in df['tags'] for item in sublist)

for value in unique_tags:
    df[value] = df['tags'].apply(lambda x: value in x if x else False)

df = df.drop(columns=['tags'])

#### non_mercado_pago_payment_methods

First, we retrieve all the unique keys present in the dictionaries within the column 'non_mercado_pago_payment_methods'

In [18]:
print(f"Unique keys in non_mercado_pago_payment_methods: {column_unique_keys(df['non_mercado_pago_payment_methods'],True)}")

Unique keys in non_mercado_pago_payment_methods: {'id', 'type', 'description'}


It seems that 'description' is the key that provides us with information about the available payment methods for purchasing a product

In [19]:
# STEP 06: Get the description of the available payment methods.
df['non_mercado_pago_payment_methods'] = df['non_mercado_pago_payment_methods'].apply(lambda x: [d.get('description') for d in x] if x else [])

Unpack the 'non_mercado_pago_payment_methods' into boolean columns and then remove the 'non_mercado_pago_payment_methods' column.

In [20]:
# STEP 07: Unpack non_mercado_pago_payment_methods in boolean columns and delete non_mercado_pago_payment_methods
unique_payments = set(item for sublist in df['non_mercado_pago_payment_methods'] for item in sublist)

for value in unique_payments:
    df[value] = df['non_mercado_pago_payment_methods'].apply(lambda x: value in x if x else False)

df = df.drop(columns=['non_mercado_pago_payment_methods'])

#### pictures

First, we retrieve all the unique keys present in the dictionaries within the column 'pictures'

In [21]:
print(f"Unique keys in pictures: {column_unique_keys(df['pictures'],True)}")

Unique keys in pictures: {'max_size', 'quality', 'size', 'secure_url', 'url', 'id'}


None of the keys present in the dictionaries seem to contain relevant information, and several of them are devoid of any data. However, a potential analysis could involve examining the number of images associated with each product.

In [22]:
# STEP 08: Get the number of pictures of the product.
df['num_pictures']  = df['pictures'].apply(lambda x: len(x) if isinstance(x, list) else None)

In [23]:
# STEP 09: Delete pictures.
df = df.drop(columns=["pictures"])

#### variations

First, we retrieve all the unique keys present in the dictionaries within the column 'variations'

In [24]:
print(f"Unique keys in variations: {column_unique_keys(df['variations'],True)}")

Unique keys in variations: {'price', 'attribute_combinations', 'seller_custom_field', 'available_quantity', 'sold_quantity', 'picture_ids', 'id'}


The 'variations' column seems to contain irrelevant information, with the majority of its data being represented by None values.

In [25]:
# STEP 10: Delete variations
df = df.drop(columns=["variations"])

#### attributes

First, we retrieve all the unique keys present in the dictionaries within the column 'attributes'

In [26]:
print(f"Unique keys in attributes: {column_unique_keys(X_train['attributes'],True)}")

Unique keys in attributes: {'value_name', 'value_id', 'attribute_group_name', 'attribute_group_id', 'name', 'id'}


The 'attributes' column seems to contain irrelevant information, with the majority of its data being represented by None values.

In [27]:
# STEP 11: Delete attributes
df = df.drop(columns=["attributes"])

### Empty Columns

We update the types of objects present in each column of the dataframe.

In [28]:
types_df = categorize_data_types(df)

We conduct an analysis of string-composed columns to detect empty strings, those solely comprising spaces, or numeric strings, aiming to enhance data usability through transformation

In [29]:
# STEP 11: Cast str columns
columns_w_str = types_df[types_df['basic_types'].str.contains("str")].index.tolist()
print(f"str columns:\t",columns_w_str)

preprocess_str_columns(df, columns_w_str)

str columns:	 ['warranty', 'sub_status', 'condition', 'deal_ids', 'site_id', 'listing_type_id', 'buying_mode', 'listing_source', 'parent_item_id', 'category_id', 'last_updated', 'international_delivery_mode', 'id', 'currency_id', 'thumbnail', 'title', 'date_created', 'secure_thumbnail', 'status', 'video_id', 'permalink', 'seller_country', 'seller_state', 'seller_city', 'shipping_dimensions', 'shipping_mode', 'shipping_tags']


We check the columns of the dataframe to identify any columns containing missing data

In [30]:
# STEP 12: Get columns with missing data
missing_table = missing_values_table(df)
empty_columns = missing_table.index.tolist()
missing_table

,Missing Values,% of Total Values,Data Type
subtitle,90000,100.00,object
listing_source,90000,100.00,object
coverage_areas,90000,100.00,object
shipping_methods,90000,100.00,object
differential_pricing,90000,100.00,object
catalog_product_id,89993,99.99,float64
shipping_dimensions,89978,99.98,object
shipping_tags,89941,99.93,object
original_price,89870,99.86,float64
deal_ids,89783,99.76,object


The 'warranty', 'parent_item_id' and 'descriptions' columns will be evaluated further later on, while the rest of the columns will be removed due to their high percentage of missing values

In [31]:
# STEP 13: Delete empty columns
empty_columns_expeptions = ["warranty", "parent_item_id", "descriptions", "thumbnail", "secure_thumbnail", "seller_country", "seller_state", "seller_city"]

for column in empty_columns_expeptions:
    empty_columns.remove(column)

print(f"Columns to remove due to high count of missing data: {empty_columns}")
print(f"Columns to be analyzed due to the presence of missing data: {empty_columns_expeptions}")

for column in empty_columns:
    df = df.drop(columns=[column])

Columns to remove due to high count of missing data: ['subtitle', 'listing_source', 'coverage_areas', 'shipping_methods', 'differential_pricing', 'catalog_product_id', 'shipping_dimensions', 'shipping_tags', 'original_price', 'deal_ids', 'official_store_id', 'sub_status', 'video_id', 'shipping_free_methods']
Columns to be analyzed due to the presence of missing data: ['warranty', 'parent_item_id', 'descriptions', 'thumbnail', 'secure_thumbnail', 'seller_country', 'seller_state', 'seller_city']


#### Columns With Missing data

#### descriptions

First, we retrieve all the unique keys present in the dictionaries within the column 'descriptions'

In [32]:
print(f"Unique keys in descriptions: {column_unique_keys(df['descriptions'])}")

Unique keys in descriptions: {'id'}


Let's compare the information stored in the 'descriptions' column with that stored in the 'id' column.

In [33]:
random_row = random.randint(0, df.shape[0])
print(f"Information for row {random_row} in column id: {df['id'].iloc[random_row]}, and in column descriptions: {df['descriptions'].iloc[random_row]['id']}")

Information for row 50198 in column id: MLA1923708022, and in column descriptions: MLA1923708022-906724134


Considering the high similarity between the information in descriptions and that in id, descriptions is removed to avoid redundancy and streamline the dataset.

In [34]:
# STEP 14: Delete descriptions
df = df.drop(columns=['descriptions'])

#### warranty

Extract the unique values from the column

In [35]:
print(f"Number of unique values: {len(df['warranty'].value_counts())}")

Number of unique values: 9535


Given its extensive range of categories and considerable missing data, the decision has been made to discard this column. The creation of the new column "warranty_info" serves to distill complex warranty information into a simplified binary variable, capturing whether any warranty details are provided. This transformation streamlines the feature space while retaining crucial insights about the presence or absence of warranty information. Moreover, it acknowledges the tendency for new items to frequently disclose warranty duration or status, contrasting with the relatively rare occurrence of warranty information for used products.

In [36]:
# STEP 15: Create warranty_info and delete descriptions
df['warranty_info'] = df['warranty'].apply(lambda x: True if x is not None else False)
df = df.drop(columns=["warranty"])

Now that we've handled columns containing lists and dictionaries, as well as empty columns, let's shift our attention to columns composed of basic Python types, we will work with groups of related columns.

### Address Columns

Obtain the count of unique values for each column.

In [37]:
address_columns = ["seller_country", "seller_state", "seller_city"]

for column in address_columns:
    print(f"Unique values of {column}: {len(df[column].unique())}")

Unique values of seller_country: 2
Unique values of seller_state: 25
Unique values of seller_city: 3480


In [38]:
df["seller_country"].value_counts()

Argentina    89999
Name: seller_country, dtype: int64

In [39]:
df["seller_state"].value_counts()

Capital Federal        52143
Buenos Aires           31482
Santa Fe                2398
Córdoba                 1727
Mendoza                  400
Chubut                   335
Entre Ríos               249
Tucumán                  214
San Juan                 132
Salta                    131
Misiones                 122
Río Negro                119
Corrientes               110
Neuquén                   87
La Pampa                  70
Chaco                     69
San Luis                  56
Jujuy                     33
Formosa                   28
Santiago del Estero       26
Santa Cruz                22
Catamarca                 20
La Rioja                  16
Tierra del Fuego          10
Name: seller_state, dtype: int64

Creating a new column "buenos_aires_seller" to combine sellers from Buenos Aires and the Federal Capital is justified due to their geographical proximity and socioeconomic similarities. This consolidation simplifies data analysis, ensures consistency, and facilitates meaningful comparisons. Additionally, sellers from these two regions constitute the vast majority of the dataset, making this grouping approach highly relevant and practical.

In [40]:
# STEP 16: Create buenos_aires_seller
df['buenos_aires_seller'] = df['seller_state'].isin(['Buenos Aires', 'Capital Federal'])

Given that all sellers are from Argentina, the information provided by the column seller_country is unnecessary. Furthermore, since the majority of sellers come from the Buenos Aires province or the Federal Capital, the detailed information in the seller_city column is also unnecessary, especially considering its numerous categories.

In [41]:
# STEP 17: Delete seller_country, seller_state, seller_city
for column in address_columns:
    df = df.drop(columns=[column])

### Shipping Columns

In [42]:
shipping_columns = ["shipping_mode", "free_shipping", "local_pick_up", "international_delivery_mode"]

In [43]:
print(f"Number of unique clases in shipping_mode: {list(df['shipping_mode'].unique())}")

Number of unique clases in shipping_mode: ['not_specified', 'me2', 'custom', 'me1']


We unpack the 'shipping_mode' column into 'shipping_not_specified', 'shipping_me2', 'shipping_custom', and 'shipping_me1'.

In [44]:
# STEP 18: Unpack and delete shipping_mode
shipping_modes = list(df['shipping_mode'].unique())

for mode in shipping_modes:
    df[f'shipping_{mode}'] = df['shipping_mode'] == mode

df = df.drop(columns=['shipping_mode'])

We will check the values ​​present in the 'international_delivery_mode' column.

In [45]:
df["international_delivery_mode"].value_counts()

none    90000
Name: international_delivery_mode, dtype: int64

The 'international_delivery_mode' column has a single value, a string of 'none', so it will be removed.

In [46]:
# STEP 19: Unpack and delete shipping_mode
df = df.drop(columns=['international_delivery_mode'])

The 'free_shipping' and 'local_pick_up' columns require no further processing.

### Tag Columns

In [47]:
tag_columns = ['dragged_bids_and_visits','dragged_visits','free_relist','good_quality_thumbnail','poor_quality_thumbnail']

tags_true_counts = df[list(unique_tags)].sum(axis=0)
print("Number of True elements per column of tags:")
print(tags_true_counts)

Number of True elements per column of tags:
dragged_visits               723
good_quality_thumbnail      1537
free_relist                  259
dragged_bids_and_visits    66516
poor_quality_thumbnail        13
dtype: int64


Given the similar nature of the 'dragged_bids_and_visits' and 'dragged_visits' columns, the information will be merged into a single column named 'dragged_bids_or_visits'

In [48]:
# STEP 20: Create dragged_bids_or_visits
df["dragged_bids_or_visits"] = df["dragged_bids_and_visits"] | df["dragged_visits"]

Due to the low occurrence of True values in the 'good_quality_thumbnail', 'poor_quality_thumbnail', and 'free_relist' columns, they will be removed.

In [49]:
# STEP 21: Delete dragged_bids_and_visits, dragged_visits, free_relist, good_quality_thumbnail, poor_quality_thumbnail
for column in tag_columns:
    df = df.drop(columns=[column])

#### Payment Methods Columns

In [50]:
payment_methods_columns = ['Acordar con el comprador','American Express','Cheque certificado','Contra reembolso','Diners','Efectivo','Giro postal','MasterCard','Mastercard Maestro','MercadoPago','Tarjeta de crédito','Transferencia bancaria','Visa','Visa Electron','accepts_mercadopago']

The payment methods were grouped as follows:

- **Agree_with_buyer_payment**: Only includes the payment method "Acordar con el comprador". This method involves agreeing on the payment directly with the buyer.
  
- **Card_payment**: Includes various credit and debit card brands such as American Express, MasterCard, Visa, etc. These methods involve payment using a credit or debit card.
  
- **Cash_payment**: Includes payment methods where cash is involved, such as "Efectivo" and "Giro postal". Also includes "Contra reembolso", which typically implies cash payment upon delivery.
  
- **Bank_payment**: Includes bank-related payment methods such as "Transferencia bancaria" and "Cheque certificado". These methods involve transferring funds between bank accounts.
  
- **Mercadopago_payment**: Specifically includes the payment method "MercadoPago".

In [51]:
# STEP 22: Group the payment methods.
payment_mapping = {
    'agree_with_buyer_payment': ['Acordar con el comprador'],
    'card_payment': ['American Express', 'MasterCard', 'Mastercard Maestro', 'Tarjeta de crédito', 'Visa', 'Visa Electron', 'Diners'],
    'cash_payment': ['Efectivo', 'Giro postal', 'Contra reembolso'],
    'bank_payment': ['Transferencia bancaria', 'Cheque certificado'],
    'mercadopago_payment': ['MercadoPago']
}
for new_column, original_columns in payment_mapping.items():
    df[new_column] = df[original_columns].any(axis=1)

The column accepts_mercadopago is conveying similar information to the previously created mercadopago_payment. Therefore, the data from both columns will be merged.

In [52]:
# STEP 23: Join accepts_mercadopago and mercadopago_payment in mercadopago_payment
df["mercadopago_payment"] = df["accepts_mercadopago"] | df["mercadopago_payment"]

In [53]:
# STEP 24: Delete payment_methods_columns
for column in payment_methods_columns:
    df = df.drop(columns=[column])

### Price Columns

In [54]:
price_columns = ["currency_id", "price", "base_price"]

Performing a check on the unique values present in the 'currency_id' column

In [55]:
df["currency_id"].value_counts()

ARS    89496
USD      504
Name: currency_id, dtype: int64

The 'currency_id' column is transformed into 'price_in_usd' to indicate when the listing displays its price in USD.

In [56]:
# STEP 25: Make price_in_usd and delete currency_id
df['price_in_usd'] = df['currency_id'].replace({'USD': True, 'ARS': False})
df = df.drop(columns=['currency_id'])

'price' and 'base_price' appear to have a similar nature. An evaluation will be conducted to determine if there are significant differences in their values.

In [57]:
print(f"Number of samples where base_price and price differ: {len(df[df['price'] != df['base_price']])}")

Number of samples where base_price and price differ: 26


In [58]:
print(f"Number of inavlid values on price: {len(df[df['price']< 0])}")

Number of inavlid values on price: 0


Due to the similarity between the data in 'price' and 'base_price', one of the two columns will be removed to avoid redundancy in the data.

In [59]:
# STEP 26: Delete base_price
df = df.drop(columns=['base_price'])

### Time Columns

In [60]:
time_columns = ['start_time', 'stop_time', 'date_created', 'last_updated']

In [61]:
# STEP 27: Change columns to datetime object.
df['start_time'] = pd.to_datetime(df['start_time'], unit='ms')
df['stop_time'] = pd.to_datetime(df['stop_time'], unit='ms')
df['date_created'] = pd.to_datetime(df['date_created'], format='%Y-%m-%dT%H:%M:%S.%fZ')
df['last_updated'] = pd.to_datetime(df['last_updated'], format='%Y-%m-%dT%H:%M:%S.%fZ')

'start_time' and 'date_created' appear to have a similar nature. An evaluation will be conducted to determine if there are significant differences in their values.

In [62]:
print(f"Number of samples where start_time and date_created differ: {len(df[df['start_time'] != df['date_created']])}")

Number of samples where start_time and date_created differ: 18238


Upon thorough comparison of the columns ['start_time', 'date_created'], it was found that while there are indeed differences between the two columns, these discrepancies amount to just one second. Hence, for the purposes of this analysis, both columns effectively contain the same information.

In [63]:
df.loc[df['start_time'] != df['date_created'], ['start_time', 'date_created']].head(10)

,start_time,date_created
5,2015-08-30 14:24:01,2015-08-30 14:24:02
9,2015-10-03 23:11:29,2015-10-03 23:11:30
11,2015-09-30 15:05:20,2015-09-30 15:05:21
22,2015-10-14 15:02:18,2015-10-14 15:02:19
32,2015-09-25 18:02:07,2015-09-25 18:02:08
44,2015-09-23 13:47:26,2015-09-23 13:47:27
54,2015-09-17 14:15:35,2015-09-17 14:15:36
59,2015-09-25 14:48:16,2015-09-25 14:48:17
63,2015-09-21 17:08:42,2015-09-21 17:08:43
66,2015-10-12 19:57:48,2015-10-12 19:57:49


The corresponding columns for the week number and day of the week when the offer began, and likewise for when it ended, are created. Additionally, a column is generated to calculate the number of days that elapsed from the start to the end of the offer.

In [64]:
# STEP 28: Make start_week, start_day, stop_week, stop_day, days_active
df['start_week'] = df['start_time'].dt.isocalendar().week
df['start_day'] = df['start_time'].dt.dayofweek
df['stop_week'] = df['stop_time'].dt.isocalendar().week
df['stop_day'] = df['stop_time'].dt.dayofweek
df['days_active'] = (df['stop_time'] - df['start_time']).dt.days

The columns with redundant information are removed, and it was determined that the 'last_updated' column does not provide relevant information.

In [65]:
# STEP 29: Delete time columns
for column in time_columns:
    df = df.drop(columns=[column])

### Quantity Columns

In [66]:
quantity_columns = ["initial_quantity", "sold_quantity", "available_quantity"]

An integrity check is performed on the data for 'initial_quantity', 'sold_quantity', and 'available_quantity' columns.

In [67]:
print(f"Number of inavlid values on initial_quantity: {len(df[df['initial_quantity']< 1])}")
print(f"Number of inavlid values on sold_quantity: {len(df[df['sold_quantity']< 0])}")
print(f"Number of inavlid values on available_quantity: {len(df[df['available_quantity']< 0])}")

Number of inavlid values on initial_quantity: 0
Number of inavlid values on sold_quantity: 0
Number of inavlid values on available_quantity: 0


The 'initial_quantity', 'sold_quantity' and 'available_quantity' columns require no further processing.

### Listing Control Columns

In [68]:
listing_control_columns =["status", "buying_mode", "listing_type_id", "automatic_relist"]

We will analyze the value counts of column 'status' to gain insights into its distribution and frequency of occurrence.

In [69]:
df["status"].value_counts()

active            86116
paused             3863
closed               20
not_yet_active        1
Name: status, dtype: int64

The unpacking of the "status" column into the variables "is_active" "is_paused" "is_closed" and "is_not_yet_active" was not performed due to the limited amount of meaningful information it could provide.

In [70]:
# STEP 30: Unpack and delete status
status_values = list(df['status'].unique())
status_values.remove("not_yet_active")
for status in status_values:
    df[f'is_{status}'] = df['status'] == status

df = df.drop(columns=['status'])

We will analyze the value counts of column 'buying_mode' to gain insights into its distribution and frequency of occurrence.

In [71]:
buying_modes = df["buying_mode"].value_counts()

The unpacking of the "status" column into the variables "buy_it_now" "classified" and "auction" 

In [72]:
# STEP 31: Unpack and delete buying_mode
buying_modes = list(df['buying_mode'].unique())
for mode in buying_modes:
    df[f'mode_{mode}'] = df['buying_mode'] == mode

df = df.drop(columns=['buying_mode'])

We will analyze the value counts of column 'listing_type_id' to gain insights into its distribution and frequency of occurrence.

In [73]:
df['listing_type_id'].value_counts()

bronze          56904
free            19260
silver           8195
gold_special     2693
gold             2170
gold_premium      765
gold_pro           13
Name: listing_type_id, dtype: int64

The column listing_type_id will be unpacked to create new columns denoting each unique category present. Specifically, the categories 'gold_special', 'gold', 'gold_premium', and 'gold_pro' have been grouped under the listing_gold column. This grouping was done as these categories generally have a limited number of individual examples, allowing for better representation and generalization in the new combined column.

In [74]:
# STEP 32: Unpack and delete listing_type_id
df['listing_free'] = df['listing_type_id'] == 'free'
df['listing_bronze'] = df['listing_type_id'] == 'bronze'
df['listing_silver'] = df['listing_type_id'] == 'silver'

gold_categories = ['gold_special', 'gold', 'gold_premium', 'gold_pro']
df['listing_gold'] = df['listing_type_id'].isin(gold_categories)

df = df.drop(columns=['listing_type_id'])

The 'automatic_relist' column require no further processing.

### Links Columns

In [75]:
links_columns = ["thumbnail", "secure_thumbnail", "permalink"]

The columns 'thumbnail', 'secure_thumbnail', 'permalink' contain only links and do not contribute to the prediction task, hence they will be removed.

In [76]:
# STEP 33: Delete link columns
for column in links_columns:
    df = df.drop(columns=[column])

### Id Columns

In [77]:
id_columns = ["title","seller_id","id","parent_item_id","category_id","site_id" ]

We will analyze the unique values of the columns 'title', 'seller_id', 'id', 'parent_item_id', 'category_id' and 'site_id' to gain insights into its distribution and frequency of occurrence.

In [78]:
for i in id_columns:
    print(f"Unique values in {i}: {len(df[i].unique())}")

Unique values in title: 89008
Unique values in seller_id: 33281
Unique values in id: 90000
Unique values in parent_item_id: 69311
Unique values in category_id: 10491
Unique values in site_id: 1


All columns exhibited a large number of categories, except for site_id, which had only one unique category. Additionally, it is worth noting that seller_id values are typically assigned in the order that sellers register on the platform. Regarding other ID columns such as id, parent_item_id, category_id, and site_id, their generation function is dependent on internal policies of the company and may or may not be related to the condition of the product being new or used.

In [79]:
# STEP 34: Delete id columns
for column in id_columns:
    df = df.drop(columns=[column])

In [80]:
types_df = categorize_data_types(df)
types_df

,basic_types,others
condition,str,
price,float,
automatic_relist,bool,
initial_quantity,int,
sold_quantity,int,
available_quantity,int,
free_shipping,bool,
local_pick_up,bool,
num_pictures,int,
warranty_info,bool,


In [81]:
ordered_columns = [
    "condition",
    # Info del producto
    "price",
    "price_in_usd",
    "initial_quantity",
    "sold_quantity",
    "available_quantity",
    "warranty_info",
    # Info del modo de compra
    "mode_buy_it_now",
    "mode_classified",
    "mode_auction",
    # Info de medios de pago
    "cash_payment",
    "card_payment",
    "bank_payment",
    "mercadopago_payment",
    "agree_with_buyer_payment",
    # Info del envio
    "shipping_me1",
    "shipping_me2",
    "shipping_custom",
    "shipping_not_specified",
    "free_shipping",
    "local_pick_up",
    # Info del vendedor
    "buenos_aires_seller",
    "listing_free",
    "listing_bronze",
    "listing_silver",
    "listing_gold",
    # Info de la publicacion
    "start_week",
    "start_day",
    "stop_week",
    "stop_day",
    "days_active",
    "is_active",
    "is_paused",
    "is_closed",
    "num_pictures",
    "automatic_relist",
    "dragged_bids_or_visits",
]  

In [82]:
df = df.reindex(columns=ordered_columns)

In [83]:
df

,condition,price,price_in_usd,initial_quantity,sold_quantity,available_quantity,warranty_info,mode_buy_it_now,mode_classified,mode_auction,cash_payment,card_payment,bank_payment,mercadopago_payment,agree_with_buyer_payment,shipping_me1,shipping_me2,shipping_custom,shipping_not_specified,free_shipping,local_pick_up,buenos_aires_seller,listing_free,listing_bronze,listing_silver,listing_gold,start_week,start_day,stop_week,stop_day,days_active,is_active,is_paused,is_closed,num_pictures,automatic_relist,dragged_bids_or_visits
0,new,80.00,False,1,0,1,False,True,False,False,True,False,True,True,True,False,False,False,True,False,True,True,False,True,False,False,36,5,45,2,60,True,False,False,2,False,True
1,used,2650.00,False,1,0,1,True,True,False,False,True,False,True,True,False,False,True,False,False,False,True,True,False,False,True,False,39,5,48,2,60,True,False,False,6,False,False
2,used,60.00,False,1,0,1,False,True,False,False,True,True,True,True,False,False,True,False,False,False,True,True,False,True,False,False,37,2,45,6,60,True,False,False,1,False,True
3,new,580.00,False,1,0,1,False,True,False,False,True,True,True,True,False,False,True,False,False,False,True,True,False,False,True,False,40,0,49,4,66,True,False,False,2,False,False
4,used,30.00,False,1,0,1,True,True,False,False,True,False,True,True,True,False,False,False,True,False,True,True,False,True,False,False,35,0,43,4,60,True,False,False,2,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89995,used,68.00,False,1,0,1,True,True,False,False,False,False,False,True,False,False,True,False,False,False,False,True,False,True,False,False,40,4,49,1,60,True,False,False,1,False,True
89996,new,126.00,False,1,1,1,False,True,False,False,True,False,True,True,False,False,False,True,False,False,True,True,False,True,False,False,39,1,47,5,60,False,True,False,3,False,True
89997,new,300.00,False,1,0,1,False,True,False,False,True,False,False,True,False,False,False,False,True,False,True,True,False,True,False,False,35,0,43,4,60,True,False,False,1,False,True
89998,new,696.58,False,100,0,100,True,True,False,False,False,False,False,True,True,False,False,False,True,False,False,False,False,True,False,False,40,4,49,1,60,True,False,False,1,False,True
